In [1]:
import os
import bz2
import xml.etree.ElementTree as ET
import mwparserfromhell as mph

class Dump:
    """``Dump`` loads and parses dumps from wikipedia from
    ``path_xml`` with index ``path_idx``.
    
    Attributes
    ----------
    idx: dictionary
        ``{'page_name': (byte offset, page id, block size)}``
        Cached. Lazy.
    links: list of strings
        All links.
    article_links: list of strings
        Article links (not files, categories, etc.)
    years: list of int
        Years in the History section of a wikipedia page
        BC denoted as negative values
    page: mwparserfromhell.wikicode
        Current loaded wiki page
    path_xml: string
        Path to the zipped XML dump file.
    path_idx: string
        Path to the zipped index file.
    offset_max: int
        Maximum offset. Set as the size of the zipped dump.
    cache: xml.etree.ElementTree.Node
        Cache of the XML tree in current block
    """
    MAX_YEAR = 2020
    
    def __init__(self, path_xml, path_idx):
        self._idx = {}
        self._links = []
        self._article_links = []
        self._years = []
        self._page = None
        self.path_xml = path_xml
        self.path_idx = path_idx
        self.offset_max = 0
        self.cache = (0, None) # offset, cache
        
    @property
    def idx(self):
        if self._idx:
            return self._idx
        else:
            print('Dump: Loading index...')
            with bz2.BZ2File(self.path_idx, 'rb') as file:
                lines = [line for line in file]
            block_end = os.path.getsize(self.path_xml)
            offset_prev = block_end
            for line in reversed(lines):
                offset, pid, name = line.strip().split(b':', 2)
                offset, pid, name = (int(offset), int(pid), name.decode('utf8'))
                block_end = offset_prev if offset < offset_prev else block_end
                self._idx[name] = (offset, pid, block_end-offset)
                offset_prev = offset
            self.offset_max = max([x[0] for x in self._idx.values()])
            print('Dump: Loaded.')
            return self._idx
    
    @property
    def links(self):
        if self._links:
            return self._links
        elif self.page:
            self._links = [str(x.title) for x in self.page.filter_wikilinks()]
            self._links = [link.split('#')[0] for link in self._links]
            self._links = [link.split(' ') for link in self._links]
            self._links = [[words[0].capitalize()] + words[1:] for words in self._links]
            self._links = [' '.join(words) for words in self._links]
            return self._links
        else:
            return self._links
    
    @property
    def article_links(self):
        if self._article_links:
            return self._article_links
        elif self.links:
            self._article_links = [x for x in self.links if ':' not in x]
            return self._article_links
        else:
            return self._article_links
    
    @property
    def years(self):
        if self._years:
            return self._years
        elif self.page:
            history = Dump.get_history(self.page)
            top = self.page.get_sections()[0].strip_code()
            self._years = Dump.filter_years(top + history)
            return self._years
        else:
            return self._years
    
    @property
    def page(self):
        return self._page
    
    @page.setter
    def page(self, page):
        self._page = page
        self._links = []
        self._article_links = []
        self._years = []
    
    def load_page(self, page_name, filter_top=False):
        """Loads & returs page (``mwparserfromhell.wikicode``)
        named ``page_name`` from dump file. Returns only the
        top section if ``filter_top``.
        """
        if page_name not in self.idx.keys():
            self.page = None
            return
        offset, pid, block_size = self.idx[page_name]
        if offset == self.cache[0]:
            root = self.cache[1]
        else:
            xml = Dump.fetch_block(self.path_xml, offset, block_size)
            xml = b'<mediawiki>' + xml + b'</mediawiki>'*(offset != self.offset_max)
            root = ET.fromstring(xml)
            self.cache = (offset, root)
        text = Dump.search_id(root, pid)
        text = Dump.filter_top_section(text) if filter_top else text
        self.page = mph.parse(text, skip_style_tags = True)
        if self.page and 'REDIRECT' in self.page.strip_code():
            redirect = self.page.filter_wikilinks()[0].title
            return self.load_page(str(redirect))
        else:
            return self.page
    
    @staticmethod
    def fetch_block(path, offset, block_size):
        """ Fetches block of ``block_size`` (``int``) bytes
        at ``offset`` (``int``) in the zipped dump at 
        ``path`` (``string``) and returns the uncompressed
        text (``string``).
        """
        with open(path, 'rb') as file:
            file.seek(offset)
            return bz2.decompress(file.read(block_size))
    
    @staticmethod
    def search_id(root, pid):
        """Returns the text of the page with id ``pid``"""
        for page in root.iter('page'):
            if pid == int(page.find('id').text):
                return page.find('revision').find('text').text
    
    @staticmethod
    def filter_top_section(text):
        """Returns the top section of text,
        where the first header has the form ``==Heading==``
        """
        head = re.search(r'==.*?==', text)
        idx = head.span(0)[0] if head else len(text)
        return text[:idx] #(text[:idx], text[idx:])
    
    @staticmethod
    def get_history(page):
        """Returns the text of the history section.
        Returns ``""`` if not found.
        """
        headings = page.filter_headings()
        idx = [i for i, head in enumerate(headings) 
                       if 'History' in head or 'history' in head]
        if not idx:
            return ""
        sections = page.get_sections(include_headings=True)
        history = str(sections[idx[0]+1].strip_code())
        return history
    
    @staticmethod
    def filter_years(text):
        """Filters the years from text."""
        months = ['january', 'february', 'march', 'april', 'may', 'june',
                  'july', 'august', 'september', 'october', 'november', 'december']
        prepositions = ['around', 'after', 'at', 'as',
                        'approximately', 'before', 'between', 'by',
                        'during', 'from', 'in', 'near', 'past',
                        'since', 'until', 'within'] # removed: about, on
        conjugations = ['and']
        articles = ['the']
        times = ['early', 'mid', 'late']
        patterns = months + prepositions + conjugations + articles + times
        re_string = r'\b(' + '|'.join(patterns) + r')\b(\s|-)\b([0-9]{3,4})s?\b(?i)(?!\sMYA)\s?(BCE|BC)?'
        years = [int(match.group(3)) * (-2*bool(match.group(4))+1)
                for match in re.finditer(re_string, text, re.IGNORECASE)]
        re_string = r'([0-9]{1,2})(st|nd|rd|th) century\s?(BCE|BC)?'
        centuries = [(int(match.group(1)) * 100 - 100) * (-2*bool(match.group(2))+1)
                     for match in re.finditer(re_string, text, re.IGNORECASE)]
        years += centuries
        years = [y for y in years if y<Dump.MAX_YEAR]
        return sorted(years + centuries)
    
def get_links(dump, page_name, filter_top = False):
    if page_name not in dump.idx.keys():
        return 
    offset, pid, block_size = dump.idx[page_name]
    xml = Dump.fetch_block(path_xml, offset, block_size)
    xml = b'<mediawiki>' + xml + b'</mediawiki>'*(offset != 0)
    root = ET.fromstring(xml)
    text = Dump.search_id(root, pid)
    text = Dump.filter_top_section(text) if filter_top else text
    page = mph.parse(text, skip_style_tags = True)
    links = [str(x.title) for x in page.filter_wikilinks()]
    return links

In [2]:
import sys
import pandas as pd
import codecs
import copy
import re
import numpy as np
import scipy
from scipy.spatial.distance import cosine
from scipy.io import savemat
import networkx as nx

def clean_entity_name(name):
    
    name = name.replace('/wiki/', '') # remove leading address sub-string
    name = name.split("_")
    processed_string = []
    for token in name:
        try:
            token_copy = copy.deepcopy(token)
            # Find and replace six characters long encodings (they are divided in pairs by % signs)
            raw_decoding_tags_1 = re.findall('%[a-zA-Z\d]{2}%[a-zA-Z\d]{2}%[a-zA-Z\d]{2}', token_copy)
            processed_decoding_tags_1 = {}
            for idx, tag in enumerate(raw_decoding_tags_1):
                tag = tag.replace('%', '')
                tag = tag.lower()
                decoding = codecs.decode(tag, "hex").decode('utf-8')
                processed_decoding_tags_1[raw_decoding_tags_1[idx]] = decoding
            for key, value in processed_decoding_tags_1.items():
                token_copy = token_copy.replace(key, value) 
            # Find and replace four characters long encodings
            raw_decoding_tags_2 = re.findall('%[a-zA-Z\d]{2}%[a-zA-Z\d]{2}', token_copy)
            processed_decoding_tags_2 = {}
            for idx, tag in enumerate(raw_decoding_tags_2):
                tag = tag.replace('%', '')
                tag = tag.lower()
                decoding = codecs.decode(tag, "hex").decode('utf-8')
                processed_decoding_tags_2[raw_decoding_tags_2[idx]] = decoding
            for key, value in processed_decoding_tags_2.items():
                token_copy = token_copy.replace(key, value)
            # Find and replace two characters long encodings
            raw_decoding_tags_3 = re.findall('%[a-zA-Z\d]{2}', token_copy)
            processed_decoding_tags_3 = {}
            for idx, tag in enumerate(raw_decoding_tags_3):
                tag = tag.replace('%', '')
                tag = tag.lower()
                decoding = codecs.decode(tag, "hex").decode('utf-8')
                processed_decoding_tags_3[raw_decoding_tags_3[idx]] = decoding
            for key, value in processed_decoding_tags_3.items():
                token_copy = token_copy.replace(key, value)
            processed_string.append(token_copy)
        except UnicodeDecodeError:
            print('Could not process %s.' % token)
            processed_string.append(token_copy)

    return ' '.join(processed_string)

# test_string = 'Mosque%E2%80%93Cathedral_of_C%C3%B3rdoba'
# clean_entity_name(test_string)

# test_string = 'Bah%C3%A1%27%C3%AD_Faith'
# clean_entity_name(test_string)

# test_string = '/wiki/99_and_44/100%25_Dead'
# clean_entity_name(test_string)

# test_string = 'Camden%2C_New_Jersey'
# clean_entity_name(test_string)

# test_string = 'Palu,_Elaz%C4%B1%C4%9F'
# clean_entity_name(test_string)

In [3]:
path_base = '/Users/sppatankar/Developer/My Passport/Curiosity/Data/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = Dump(path_xml, path_index)

In [4]:
wiki_df = pd.read_csv(path_base + 'KNOT_data_raw.csv')
wiki_df.head(3)

,ID,SourceName,TargetName,Day,TimeOrder,Hyperlink,DistanceWeights,AgeYears,SexOrient,Race,GenderFactor,EducDeg,Income,JE_5D,DS_5D,ST_5D,SC_5D,TS_5D,Count,Weight
0,101,/wiki/Jeff_Bezos,/wiki/Cloud_infrastructure,1,1,no,1.0,23.27945,Heterosexual,AsiaAm,0,BachDegree,20to49k,4.4,4.25,1.6,2.8,2.0,1,0.0
1,101,/wiki/Cloud_infrastructure,/wiki/Cloud_computing_security,1,2,yes,0.2,23.27945,Heterosexual,AsiaAm,0,BachDegree,20to49k,4.4,4.25,1.6,2.8,2.0,2,0.8
2,101,/wiki/Cloud_computing_security,/wiki/Cloud_infrastructure,1,3,no,0.2,23.27945,Heterosexual,AsiaAm,0,BachDegree,20to49k,4.4,4.25,1.6,2.8,2.0,3,0.8


In [5]:
# create UID for each page
source_nodes = set(wiki_df['SourceName'].tolist())
target_nodes = set(wiki_df['TargetName'].tolist())
source_nodes.update(target_nodes)
node_set = {entity: name for name, entity in enumerate(source_nodes)}
wiki_df['SourceUID'] = wiki_df['SourceName'].apply(lambda x: node_set[x])
wiki_df['SrcNameClean'] = wiki_df['SourceName'].apply(lambda x: clean_entity_name(x))
wiki_df['TargetUID'] = wiki_df['TargetName'].apply(lambda x: node_set[x])
wiki_df['TgtNameClean'] = wiki_df['TargetName'].apply(lambda x: clean_entity_name(x))
wiki_df.head(3)

Could not process Elaz%C4%B1%C4%9F.
Could not process Dav%C3%AD%C3%B0sd%C3%B3ttir.
Could not process %C3%9E%C3%B3risd%C3%B3ttir.
Could not process %C4%90%C3%ACnh.
Could not process Dvo%C5%99%C3%A1k.
Could not process Elaz%C4%B1%C4%9F.
Could not process Dav%C3%AD%C3%B0sd%C3%B3ttir.
Could not process %C3%9E%C3%B3risd%C3%B3ttir.
Could not process %C4%90%C3%ACnh.
Could not process Dvo%C5%99%C3%A1k.


,ID,SourceName,TargetName,Day,TimeOrder,Hyperlink,DistanceWeights,AgeYears,SexOrient,Race,...,DS_5D,ST_5D,SC_5D,TS_5D,Count,Weight,SourceUID,SrcNameClean,TargetUID,TgtNameClean
0,101,/wiki/Jeff_Bezos,/wiki/Cloud_infrastructure,1,1,no,1.0,23.27945,Heterosexual,AsiaAm,...,4.25,1.6,2.8,2.0,1,0.0,13476,Jeff Bezos,4314,Cloud infrastructure
1,101,/wiki/Cloud_infrastructure,/wiki/Cloud_computing_security,1,2,yes,0.2,23.27945,Heterosexual,AsiaAm,...,4.25,1.6,2.8,2.0,2,0.8,4314,Cloud infrastructure,13936,Cloud computing security
2,101,/wiki/Cloud_computing_security,/wiki/Cloud_infrastructure,1,3,no,0.2,23.27945,Heterosexual,AsiaAm,...,4.25,1.6,2.8,2.0,3,0.8,13936,Cloud computing security,4314,Cloud infrastructure


In [6]:
len(source_nodes)

18378

In [7]:
# 5 out of 18378 nodes could not be properly parsed

In [8]:
# split data by individual
ID_groups = wiki_df.groupby('ID')
for subj_num, (ID, group) in enumerate(ID_groups):
    print('Subject %d of %d.' % (subj_num, len(ID_groups)))
    # enforce time ordering
    group.sort_values(by = ['TimeOrder'], inplace = True)
    # network_df = group[['TimeOrder', 'SourceUID', 'SrcNameClean', 'TargetUID', 'TgtNameClean', 'Hyperlink']].reset_index(drop = True)
    G = nx.Graph()
    node_list = []
    for index, row in group.iterrows():
        from_node = row['SrcNameClean']
        node_list.append(from_node)
        G.add_node(from_node)
        to_node = row['TgtNameClean']
        node_list.append(to_node)
        G.add_node(to_node)
        # G.add_edge(from_node, to_node) # this adds edges actually traversed
    # Add hyperlinked edges 
    pages = list(set(node_list))
    for page_1 in pages:
        page_1_links = get_links(dump, page_1)
        if page_1_links is not None:
            page_1_links = [x.lower() for x in page_1_links]
            for page_2 in pages:
                if page_2.lower() in page_1_links:
                    G.add_node(page_2) # this should be redundant 
                    # print('from: %s, to: %s, LINK' % (page_1, page_2))
                    G.add_edge(page_1, page_2)
                    # print('LINK Network Size: n = %d, e = %d.' % (len(G.nodes()), len(G.edges())))
    adj_G = nx.linalg.graphmatrix.adjacency_matrix(G, weight = 'weight')
    filename = 'subj_' + str(ID) + '.mat'
    mdic = {'subj': ID, 'adj': scipy.sparse.csr_matrix.todense(adj_G), 
          'nodes': list(G.nodes())}
    savemat(filename, mdic)

Subject 0 of 149.
Dump: Loading index...


/Users/sppatankar/opt/anaconda3/envs/everything/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Dump: Loaded.
Subject 1 of 149.


/Users/sppatankar/opt/anaconda3/envs/everything/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Subject 2 of 149.


...

/Users/sppatankar/opt/anaconda3/envs/everything/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Subject 147 of 149.


/Users/sppatankar/opt/anaconda3/envs/everything/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Subject 148 of 149.


/Users/sppatankar/opt/anaconda3/envs/everything/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
